<a href="https://colab.research.google.com/github/jadechip/Talk2Task/blob/main/example.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [15]:
!pip -q install openai

In [6]:
import re
import os
import subprocess
import openai

In [12]:
class Tool:
    def __init__(self, name):
        self.name = name

    def useTool(self, command):
        pass

class CommandExecutor(Tool):
    def __init__(self):
        super().__init__('CommandExecutor')

    def useTool(self, command):
        result = subprocess.run(command, capture_output=True, shell=True)
        return result.stdout

class FileCreator(Tool):
    def __init__(self):
        super().__init__('FileCreator')

    def useTool(self, command):
        os.makedirs(command, exist_ok=True)
        return f"Directory {command} created."

class CodeWriter(Tool):
    def __init__(self, filename):
        super().__init__('CodeWriter')
        self.filename = filename

    def useTool(self, code):
        with open(self.filename, 'w') as file:
            file.write(code)
        return f"File {self.filename} written."

class Talk2Task:
    def __init__(self, api_key):
        self.api_key = api_key
        openai.api_key = api_key
        self.tool_map = {
            'CommandExecutor': CommandExecutor(),
            'FileCreator': FileCreator(),
        }
        self.tool_prompts = {
            'CommandExecutor': 'CommandExecutor allows you to run shell commands. Use it by writing [CommandExecutor(your_command)].',
            'FileCreator': 'FileCreator lets you create a directory. Use it by writing [FileCreator(/path/to/directory)].',
        }

    def generate_llm_messages(self, query):
        tool_descriptions = '\n'.join([f'{name}: {desc}' for name, desc in self.tool_prompts.items()])
        messages = [
            {
                "role": "system",
                "content": f"""
                Based on the tools available to you ({tool_descriptions}), break down a given task or query into specific steps. 
                For each step, specify the tool that should be used and the exact command that should be executed.
                Your response MUST be a JSON list of commands in the following format:
                [["ToolName", "Command1"], ["ToolName", "Command2"], ...]
                Do not provide any additional text or explanation in the response, only the JSON list of commands.
                Query:
                {query}
                """
            },
            {
                "role": "user",
                "content": query
            }
        ]
        return messages

    def interact_with_gpt(self, prompt):
        messages = self.generate_llm_messages(prompt)
        completion = openai.ChatCompletion.create(
            model="gpt-3.5-turbo",
            temperature=0.2,
            messages=messages
        )
        response = completion.choices[0].message['content']
        print(f"Response: {response}")

        return response

    def parse_response(self, response):
        response = self.preprocess_response(response)

        patterns = [
            r'\["(\w+)"\s*,\s*"([^"]+)"\]',
            r'\[(\w+)\((.+)\)\]',
        ]

        for pattern in patterns:
            matches = re.findall(pattern, response)
            if matches:
                break
        else:
            raise ValueError("Response does not contain a list of commands")

        commands = [(match[0], match[1]) for match in matches]

        return commands

    def preprocess_response(self, response):
        response = response.replace('\n', '')
        response = re.sub(r'\s+', ' ', response)
        return response.strip()

    def execute_commands(self, commands):
        results = []
        current_file = None
        for cmd in commands:
            tool, args = cmd
            if tool in self.tool_map:
                print(f"Executing command: {tool}({args})")
                result = self.tool_map[tool].useTool(args)
                print(f"Result: {result}\n")
                results.append((tool, args, result))
            else:
                raise Exception(f'Unknown command type: {tool}')
        return results

    def process_prompt(self, prompt, retry=3):
        try:
            response = self.interact_with_gpt(prompt)
            commands = self.parse_response(response)
            results = self.execute_commands(commands)
            return results
        except Exception as e:
            if retry > 0:
                print(f"Error encountered: {e}. Retrying...")
                return self.process_prompt(prompt, retry - 1)
            else:
                raise e


In [17]:
# Initialize the library with your OpenAI API key
lib = Talk2Task("<OPENAI_API_KEY>")

# Define a prompt that GPT-X will understand to create a directory
prompt = """
Create a phaser.js game called Turing Town with the following gameplay mechanics:

Starting a round: At the start of each round, the server should randomly assign roles to each connected player. These roles could be stored in memory or in a database, and each player's client should be notified of their role.

Example code:
roles = ['AI', 'Human']
players_roles = {player: random.choice(roles) for player in players}
Controlling avatars: Each player should have control over their avatar. For human players, this would involve sending movement and action commands from the client to the server. For AI players, the server should periodically send commands based on the AI's behavior.

Example code:
// Example of a human player sending a move command
socket.emit('move', { direction: 'up' });

// Example of the server moving an AI player
if (player.role === 'AI') {
  let action = AI.getAction(player);
  movePlayer(player, action);
}
Interactions and actions: The server needs to handle various interactions and actions, such as interacting with objects or communicating with other avatars. These could be handled as different types of commands sent from the client.

Example code:
// Example of a player interacting with an object
socket.emit('interact', { object: 'paintbrush' });
Decision making: Players should be able to vote on who they think is human. The server should handle these votes and keep track of them.

Example code:
// Example of a player voting
socket.emit('vote', { suspect: 'player2' });
Scoring: The server should calculate scores based on the rules you described. After each round, it should send the scores to each client.

Example code:
calculate_scores(players, votes)
AI learning: After each round, the AI should update its behavior based on the experiences of that round. This would involve integrating your AI models with the game server.
python
Copy code
for player in players:
  if player.role === 'AI':
    player.AI_model.update(experiences)
"""

# Process the prompt
result = lib.process_prompt(prompt)

# Print the result
print(result)

Response: [["CommandExecutor", "npm install phaser"], 
["CommandExecutor", "npm install socket.io"], 
["CommandExecutor", "npm install express"], 
["FileCreator", "/path/to/TuringTown"], 
["CommandExecutor", "cd /path/to/TuringTown && phaser init"], 
["CommandExecutor", "cd /path/to/TuringTown && npm init"], 
["CommandExecutor", "cd /path/to/TuringTown && npm install"], 
["CommandExecutor", "cd /path/to/TuringTown && npm install nodemon"], 
["CommandExecutor", "cd /path/to/TuringTown && npm install body-parser"], 
["CommandExecutor", "cd /path/to/TuringTown && npm install cors"], 
["CommandExecutor", "cd /path/to/TuringTown && npm install mongoose"], 
["CommandExecutor", "cd /path/to/TuringTown && npm install dotenv"], 
["CommandExecutor", "cd /path/to/TuringTown && npm install bcrypt"], 
["CommandExecutor", "cd /path/to/TuringTown && npm install jsonwebtoken"], 
["CommandExecutor", "cd /path/to/TuringTown && npm install axios"], 
["CommandExecutor", "cd /path/to/TuringTown && npm inst